In [1]:
import tensorflow as tf
#from tensorflow.keras.layers import *
#from tensorflow.keras.models import Model
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)
#tf.keras.backend.set_epsilon(1e-4)


import keras
from keras import backend as K
from keras.layers import *

from bert import BertModelLayer

import bert
import os

Using TensorFlow backend.


In [10]:
max_len = 384

bert_layer = None

pTrain_dir = 'cased_L-12_H-768_A-12'

def bLayer():
    global bert_layer
    
    pTrain_dir = 'cased_L-12_H-768_A-12'
    
    bert_params = bert.params_from_pretrained_ckpt(pTrain_dir)

    bert_layer = bert.BertModelLayer.from_params(bert_params, name="bert")

    bert_layer.apply_adapter_freeze()
    
    bert_layer.trainable = False
    
bLayer()
def loadBertCheckpoint():
    pTrain_dir = pTrain_dir
    checkpointName = os.path.join(pTrain_dir, "bert_model.ckpt")

    bert.load_stock_weights(bert_layer, checkpointName)
    
    
def model():
    
    #model = tf.keras.Sequential()

    '''model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(max_len,), name='input'),
        bLayer,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation='tanh',dtype='float32')
    ])
    model.build(input_shape=(None, max_seq_length))

    return model'''
    
    i = tf.keras.layers.Input(shape = (max_len,), name ='input')
    bertLayer = bert_layer(i)
    flat = tf.keras.layers.Flatten()(bertLayer)
    #drop1 = tf.keras.layers.Dropout(.2)(flat)
    dense1 = tf.keras.layers.Dense(16,activation='relu')(flat)
    drop1 = tf.keras.layers.Dropout(.1)(dense1)
    dense1_1 = tf.keras.layers.Dense(16,activation ='relu')(drop1)
    dense2 = tf.keras.layers.Dense(2,activation='tanh')(dense1_1)
    
    output = tf.keras.layers.Dense(1, activation = 'tanh', dtype='float32')(dense2)
    
    model = tf.keras.models.Model(inputs=i, outputs = output)
    
    return model
mod = model()

opt = tf.keras.optimizers.Adam(0.000001)

mod.compile(loss=tf.keras.losses.mean_squared_error,
              optimizer= opt,
              metrics=['categorical_accuracy','mean_absolute_error'])

print(mod.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 384)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 384, 768)          107718144 
_________________________________________________________________
flatten_1 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                4718608   
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 34  

In [3]:
import pandas as pd
from bert import bert_tokenization

vocab= pTrain_dir+'/vocab.txt'

df = pd.read_csv('news_text_2.csv')
df.head()

text_ex = df.iloc[0].Text
print(text_ex)

tokenizer = bert_tokenization.FullTokenizer(vocab_file=vocab, do_lower_case=False)

t = tokenizer.tokenize(text_ex)

print(t)
print(len(t))

print(len(df))

US disrupted alleged Russian trolls' internet access during midterms Report Cyber Command chief had signaled more aggressive moves outside U.S. borders.U.S. cyber operators disrupted internet access for purported Russian trolls on the day of the 2018 midterm elections in a bid to hamper their propaganda operations, according to a report by The Washington Post.The operation by U.S. Cyber Command reportedly targeted the St. Petersburgbased Internet Research Agency IRA, which U.S. officials and prosecutors from Robert Muellers special counsel team allege operated an expansive influence operation designed in part to undermine American democracy by sowing discord online ahead of the 2016 presidential election and 2018 midterm elections.They basically took the IRA offline, an individual familiar with the operation told the Post. They shut em down.The head of U.S. Cyber Command and the National Security Agency NSA, Gen. Paul Nakasone, previously indicated his cyber warriors would be more aggr

In [4]:
X=np.load("train_text.npy", allow_pickle = True)
Y=np.load("train_bias.npy")
X_val=np.load("val_text.npy", allow_pickle = True)
Y_val=np.load("val_bias.npy")

In [5]:
#clean data

clean_X = []
clean_Y = []

clean_val_X = []
clean_val_Y = []

for i in range(len(X)):  
    if X[i] is None:
        print('none')
        
    if type(X[i]) is float:
        print('a')
        print(i)
    else:
        clean_X.append(X[i])
        clean_Y.append(Y[i])
        
        if len(X[i]) <5:
            print('nan')
            
for i in range(len(X_val)):  
    if X_val[i] is None:
        print('none')
        
    if type(X_val[i]) is float:
        print('a')
        print(i)
    else:
        clean_val_X.append(X_val[i])
        clean_val_Y.append(Y_val[i])
        
        if len(X_val[i]) <5:
            print('nan')
            
print(len(X))
print(len(clean_X))

X = clean_X
Y = clean_Y
X_val = clean_val_X
Y_val = clean_val_Y

a
1792
a
2248
a
442
a
619
3021
3019


In [6]:
from tqdm import tqdm

def to_token(X):
    for i in tqdm(range(len(X))):
        
        tmp=tokenizer.tokenize(X[i])[:383]
        
        
        a = ['[CLS]']
        
        tmp = a+ tmp
        
        while len(tmp)<384:
            tmp.append('0')
            
        tmp = tokenizer.convert_tokens_to_ids(tmp)

    
        X[i]=np.array(tmp)
        
    X = np.array(X)
    return X

X = to_token(X)
X_val = to_token(X_val)
print(X[0])
print(X_val[0])

100%|███████████████████████████████████████████████████████████████████████████████| 754/754 [00:04<00:00, 186.32it/s]

[  101  1168 14255 18236 23193  1116  1164  1147  2491   119   146  5548
 15021  1103  4420   117  1131  1500  1103  6962   119  1337  1110  1136
  1541  1226  1104  1139  2261  6136   117  1133   146  1176  4518  1113
   170  1437  1111  1172   119  1153  1156  2842  1213   117  1587 13948
  1106  2073  2807   191 24874  1233  1120 21685  1116   117  2222  1106
 18456  1146  1137 16989  2490  1121  1103  2489  1105 13265  1115  4303
  4405  1172   119  1130   170  1581  2025  1521  1118  1103  6962   117
  1330 16408 12223 10359  1758  9374  1103  1269  1395  1160  1551  1107
  1546  1106  7166  1103  1713  1104   170 13713  3554  1401   119  1706
  1199   117  1103  7279  1547  3166  5119  1409  1128  1267  1240  2261
  1112  9523  1103  4809   117  1897  1190  1198   188 26580  6791  1146
  6477  1279   117  1240  1162  2620  1106  1138   170  6353  2305  1104
  3007  7747  1128  6387  1103   182  4184   119  1252  1184  1116  9495
  1110  1293  1374 19328  1116  3166  1106  1138  4

In [7]:
n = 4
print(X[n])
print(len(X[n]))

[  101  4461  5481  1163  1157  1148 13284 27618  5022  2204   130  1121
   170  1214  2403   117  1855  1118  1103  1269  6157   188 15363  1346
  1107  1103  1214  1115  2644  1168  6250  1715  9780   119  1109  3085
  6310   170  5022  1104   123   119   125  3775   117  1137   122   119
  3614   170  2934   117  1113  7143  1104  1275   119   124  3775   119
  2695  1132  2211  1190  1103  1269  1669   170  1214  2206   117  1165
  1103  3016  2829   123   119   128  3775   117  1137   122   119  2532
   170  2934   117  1113  1647 21081  7143  1104  1429   119   122  3775
   119  1398   119   119   119   121   121   121   121   121   121   121
   121   121   121   121   121   121   121   121   121   121   121   121
   121   121   121   121   121   121   121   121   121   121   121   121
   121   121   121   121   121   121   121   121   121   121   121   121
   121   121   121   121   121   121   121   121   121   121   121   121
   121   121   121   121   121   121   121   121   

In [11]:
print(X.shape)

mod.fit(X,np.array(Y),epochs = 10, validation_data =(X_val,np.array(Y_val)), batch_size = 2)

(3019, 384)
Train on 3019 samples, validate on 754 samples
Epoch 1/10
3019/3019 [==============================] - 351s 116ms/sample - loss: 0.1099 - categorical_accuracy: 1.0000 - mean_absolute_error: 0.2498 - val_loss: 0.0928 - val_categorical_accuracy: 1.0000 - val_mean_absolute_error: 0.2219
Epoch 2/10
3019/3019 [==============================] - 329s 109ms/sample - loss: 0.0975 - categorical_accuracy: 1.0000 - mean_absolute_error: 0.2306 - val_loss: 0.0908 - val_categorical_accuracy: 1.0000 - val_mean_absolute_error: 0.2178
Epoch 3/10
3019/3019 [==============================] - 329s 109ms/sample - loss: 0.0944 - categorical_accuracy: 1.0000 - mean_absolute_error: 0.2256 - val_loss: 0.0895 - val_categorical_accuracy: 1.0000 - val_mean_absolute_error: 0.2152
Epoch 4/10
3019/3019 [==============================] - 328s 109ms/sample - loss: 0.0925 - categorical_accuracy: 1.0000 - mean_absolute_error: 0.2235 - val_loss: 0.0890 - val_categorical_accuracy: 1.0000 - val_mean_absolute_err